In [3]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [4]:
data = pd.read_csv("jester-data-1.csv", header=None)

In [5]:
n_features = 2

#user_ratings = data.values
user_ratings = data.values[0:100]
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [6]:
latent_item_features

array([[0.925, 0.981],
       [0.081, 0.424],
       [0.3  , 0.825],
       [0.727, 0.679],
       [0.275, 0.19 ],
       [0.589, 0.529],
       [0.97 , 0.358],
       [0.768, 0.139],
       [0.027, 0.824],
       [0.471, 0.261],
       [0.13 , 0.505],
       [0.901, 0.994],
       [0.025, 0.243],
       [0.49 , 0.188],
       [0.982, 0.348],
       [0.165, 0.867],
       [0.85 , 0.236],
       [0.082, 0.444],
       [0.879, 0.304],
       [0.67 , 0.245],
       [0.931, 0.257],
       [0.209, 0.636],
       [0.738, 0.962],
       [0.937, 0.962],
       [0.43 , 0.209],
       [0.282, 0.029],
       [0.715, 0.805],
       [0.42 , 0.038],
       [0.225, 0.842],
       [0.083, 0.656],
       [0.524, 0.467],
       [0.162, 0.176],
       [0.931, 0.452],
       [0.911, 0.052],
       [0.419, 0.608],
       [0.057, 0.072],
       [0.183, 0.338],
       [0.139, 0.898],
       [0.525, 0.424],
       [0.904, 0.666],
       [0.528, 0.157],
       [0.212, 0.006],
       [0.568, 0.784],
       [0.6

In [5]:
latent_user_preferences

array([[0.381, 0.327],
       [0.006, 0.216],
       [0.081, 0.524],
       [0.359, 0.905],
       [0.244, 0.99 ],
       [0.104, 0.946],
       [0.676, 0.195],
       [0.309, 0.653],
       [0.776, 0.798],
       [0.079, 0.084],
       [0.426, 0.867],
       [0.709, 0.242],
       [0.941, 0.312],
       [0.073, 0.529],
       [0.929, 0.591],
       [0.193, 0.333],
       [0.134, 0.37 ],
       [0.701, 0.501],
       [0.121, 0.792],
       [0.019, 0.719],
       [0.238, 0.237],
       [0.669, 0.226],
       [0.888, 0.095],
       [0.849, 0.995],
       [0.511, 0.243],
       [0.878, 0.86 ],
       [0.883, 0.924],
       [0.694, 0.072],
       [0.304, 0.931],
       [0.321, 0.306],
       [0.955, 0.099],
       [0.449, 0.537],
       [0.226, 0.533],
       [0.841, 0.961],
       [0.469, 0.4  ],
       [0.45 , 0.335],
       [0.017, 0.383],
       [0.275, 0.675],
       [0.833, 0.345],
       [0.105, 0.126],
       [0.245, 0.024],
       [0.527, 0.075],
       [0.717, 0.841],
       [0.4

In [7]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 100000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not np.isnan(rating)):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            print (mse)

In [ ]:
sgd()

2649.230047757464


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in subtract
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in multiply
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in square


In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda x: "(%2.3f|%2.3f)"%(x[0],x[1]))

In [ ]:
comparison_data

In [ ]:
comparison = comparison_data.to_latex()
text_file = open("comparison.txt", "w")
text_file.write(comparison)
text_file.close()